<a href="https://colab.research.google.com/github/louispaulet/Data_dependencies_and_data_integration_course/blob/main/DDDI_Project_UCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import io

In [25]:
df = pd.read_csv('https://raw.githubusercontent.com/USEPA/FDTool/master/data/input/Table1.csv')
df


,A,B,C,D,E
0,0,0,0,2,0
1,0,1,0,2,0
2,0,2,0,2,2
3,0,3,1,2,0
4,4,1,1,1,4
5,4,3,1,1,2
6,0,0,1,2,0


In [26]:
df.describe()

,A,B,C,D,E
count,7.000000,7.000000,7.000000,7.000000,7.000000
mean,1.142857,1.428571,0.571429,1.714286,1.142857
std,1.951800,1.272418,0.534522,0.487950,1.573592
min,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.500000,0.000000,1.500000,0.000000
50%,0.000000,1.000000,1.000000,2.000000,0.000000
75%,2.000000,2.500000,1.000000,2.000000,2.000000
max,4.000000,3.000000,1.000000,2.000000,4.000000


We start easy with a ucc_2() function that only checks pairs of columns

In [27]:
#create a dataframe with an "index" column that will make supersets of "test_col" all UCCs
df2 = df.copy()
df2['test_col'] = range(1, len(df) + 1)
df2

,A,B,C,D,E,test_col
0,0,0,0,2,0,1
1,0,1,0,2,0,2
2,0,2,0,2,2,3
3,0,3,1,2,0,4
4,4,1,1,1,4,5
5,4,3,1,1,2,6
6,0,0,1,2,0,7


In [28]:
def ucc_2(df):
  list_of_uccs = []
  #for every column
  for col in df:
    #for every combination of column
    for col2 in df:
      #if the columns are not the same one and they do not have duplicate rows when joined row by row:
      if (col != col2 and len(df[col].index) == len(((pd.concat([df2[col], df2[col2]], axis=1)).drop_duplicates()).index)):
        #add to the list of uccs
        list_of_uccs.append((col, col2))
  return list_of_uccs

#we run ucc() on default dataframe and get an empty result because there are no UCCs  
ucc_2(df)        

[]

In [29]:
#we can see that the function returns supersets of "test_col" as UCCs -- should we keep the symetry ?
ucc_2(df2)

[('A', 'test_col'),
 ('B', 'test_col'),
 ('C', 'test_col'),
 ('D', 'test_col'),
 ('E', 'test_col'),
 ('test_col', 'A'),
 ('test_col', 'B'),
 ('test_col', 'C'),
 ('test_col', 'D'),
 ('test_col', 'E')]

We need to find all combinations of columns that are UCC, not only two columns.
We use itertools to generate all combinations of columns from df :

In [30]:
from itertools import combinations 

column_combination = []
for i in range(1, len(df.columns)+1):
  for item in list(combinations(df.columns, i)):
    column_combination.append(item)
column_combination

[('A',),
 ('B',),
 ('C',),
 ('D',),
 ('E',),
 ('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('A', 'E'),
 ('B', 'C'),
 ('B', 'D'),
 ('B', 'E'),
 ('C', 'D'),
 ('C', 'E'),
 ('D', 'E'),
 ('A', 'B', 'C'),
 ('A', 'B', 'D'),
 ('A', 'B', 'E'),
 ('A', 'C', 'D'),
 ('A', 'C', 'E'),
 ('A', 'D', 'E'),
 ('B', 'C', 'D'),
 ('B', 'C', 'E'),
 ('B', 'D', 'E'),
 ('C', 'D', 'E'),
 ('A', 'B', 'C', 'D'),
 ('A', 'B', 'C', 'E'),
 ('A', 'B', 'D', 'E'),
 ('A', 'C', 'D', 'E'),
 ('B', 'C', 'D', 'E'),
 ('A', 'B', 'C', 'D', 'E')]

This ucc() function manages all combinations of columns

In [36]:
from IPython.display import HTML, display
import time

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))
# out = display(progress(0, 100), display_id=True)

In [37]:
def ucc(df):

  column_combination = []
  for i in range(1, len(df.columns)+1):
    for item in list(combinations(df.columns, i)):
      column_combination.append(item)

  list_of_uccs = []
  cpt = 0
  for comb in column_combination:
    frames = []
    for col_name in list(comb):
      frames.append(df[col_name])
    # print(pd.concat(frames, axis=1).drop_duplicates())
    if (len(frames[0].index) == len(((pd.concat(frames, axis=1)).drop_duplicates()).index)):
      list_of_uccs.append(comb)
    
    cpt+=1
    out.update(progress(cpt, len(column_combination)))

  return list_of_uccs

ucc(df)

[('A', 'B', 'C'),
 ('B', 'C', 'D'),
 ('B', 'C', 'E'),
 ('A', 'B', 'C', 'D'),
 ('A', 'B', 'C', 'E'),
 ('B', 'C', 'D', 'E'),
 ('A', 'B', 'C', 'D', 'E')]

Now we try it with the df2 that contains "test_col" to see if all supersets of "test_col" are generated :

In [38]:
ucc(df2)

[('test_col',),
 ('A', 'test_col'),
 ('B', 'test_col'),
 ('C', 'test_col'),
 ('D', 'test_col'),
 ('E', 'test_col'),
 ('A', 'B', 'C'),
 ('A', 'B', 'test_col'),
 ('A', 'C', 'test_col'),
 ('A', 'D', 'test_col'),
 ('A', 'E', 'test_col'),
 ('B', 'C', 'D'),
 ('B', 'C', 'E'),
 ('B', 'C', 'test_col'),
 ('B', 'D', 'test_col'),
 ('B', 'E', 'test_col'),
 ('C', 'D', 'test_col'),
 ('C', 'E', 'test_col'),
 ('D', 'E', 'test_col'),
 ('A', 'B', 'C', 'D'),
 ('A', 'B', 'C', 'E'),
 ('A', 'B', 'C', 'test_col'),
 ('A', 'B', 'D', 'test_col'),
 ('A', 'B', 'E', 'test_col'),
 ('A', 'C', 'D', 'test_col'),
 ('A', 'C', 'E', 'test_col'),
 ('A', 'D', 'E', 'test_col'),
 ('B', 'C', 'D', 'E'),
 ('B', 'C', 'D', 'test_col'),
 ('B', 'C', 'E', 'test_col'),
 ('B', 'D', 'E', 'test_col'),
 ('C', 'D', 'E', 'test_col'),
 ('A', 'B', 'C', 'D', 'E'),
 ('A', 'B', 'C', 'D', 'test_col'),
 ('A', 'B', 'C', 'E', 'test_col'),
 ('A', 'B', 'D', 'E', 'test_col'),
 ('A', 'C', 'D', 'E', 'test_col'),
 ('B', 'C', 'D', 'E', 'test_col'),
 ('A', '

In [34]:
# attributes from :
# https://archive.ics.uci.edu/ml/datasets/Forest+Fires

  #  1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
  #  2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
  #  3. month - month of the year: "jan" to "dec" 
  #  4. day - day of the week: "mon" to "sun"
  #  5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
  #  6. DMC - DMC index from the FWI system: 1.1 to 291.3 
  #  7. DC - DC index from the FWI system: 7.9 to 860.6 
  #  8. ISI - ISI index from the FWI system: 0.0 to 56.10
  #  9. temp - temperature in Celsius degrees: 2.2 to 33.30
  #  10. RH - relative humidity in %: 15.0 to 100
  #  11. wind - wind speed in km/h: 0.40 to 9.40 
  #  12. rain - outside rain in mm/m2 : 0.0 to 6.4 
  #  13. area - the burned area of the forest (in ha): 0.00 to 1090.84 
  #  (this output variable is very skewed towards 0.0, thus it may make
  #   sense to model with the logarithm transform). 

forest_fires = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv',
                        sep=',', encoding='latin-1')

forest_fires

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [39]:
ucc(forest_fires)

[]